In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
from collections import Counter
import pickle
import itertools

Choose the dataset

In [ ]:
#dataset = 'hospital'
#dataset = 'high_school'
#dataset = 'conference'
#dataset = 'workplace'
dataset = 'pr_school'

filename is the path to the datsset, like "data/primaryschool.csv"

In [13]:
time_list = []
person1_list = []
person2_list = []
for line in open(filename):
    csv_row = line.split()
    time_list.append(int(csv_row[0]))
    person1_list.append(int(csv_row[1]))
    person2_list.append(int(csv_row[2]))

Reassign nodes identities such that they start from 0 and go up to N:

In [ ]:
nodes = np.unique(person1_list+person2_list)
nodes

In [ ]:
new_nodes = np.arange(len(nodes))
new_nodes

In [ ]:
conversion = {}
for i in range(len(nodes)):
    conversion[nodes[i]] = new_nodes[i]

In [ ]:
new_person1_list = []
for node in person1_list:
    new_person1_list.append(conversion[node])
    
new_person2_list = []
for node in person2_list:
    new_person2_list.append(conversion[node])

Find interactions at every time:

In [15]:
def interaction_network_at_given_time(instant):
    edge_list = []
    for t in range(len(time_list)):
        if time_list[t] == instant:
            edge_list.append([new_person1_list[t],new_person2_list[t]])
    G=nx.Graph()
    G.add_edges_from(edge_list)
    return G

In [16]:
Dpairwise = {}
D2 = {}
D3 = {}
D4 = {}
D5 = {}

time = 0
while time < max(time_list):
    G0 = interaction_network_at_given_time(time)
    cliques = list(nx.find_cliques(G0))

    for clique in cliques:
        clique = list(np.sort(clique))
        size = len(clique)
        if size == 2:
            if tuple(clique) in D2:
                D2[tuple(clique)] += 1
            else:
                D2[tuple(clique)] = 1
        elif size == 3:
            if tuple(clique) in D3:
                D3[tuple(clique)] += 1
            else:
                D3[tuple(clique)] = 1
        elif size == 4:
            if tuple(clique) in D4:
                D4[tuple(clique)] += 1
            else:
                D4[tuple(clique)] = 1
        elif size == 5:
            if tuple(clique) in D5:
                D5[tuple(clique)] += 1
            else:
                D5[tuple(clique)] = 1
        elif size > 5:
            print(clique)
            
    pairwise_links = list(G0.edges)
    for edge in pairwise_links:
        edge = list(np.sort(edge))
        if tuple(edge) in Dpairwise:
            Dpairwise[tuple(edge)] += 1
        else:
            Dpairwise[tuple(edge)] = 1
            
    time += 20

Normalize pairwise:

In [ ]:
Dpairwise_n = {}
for key in Dpairwise:
    Dpairwise_n[key] = Dpairwise[key]/max(Dpairwise.values())

Save all pairwise:

In [18]:
dict_name = 'pairwise/%s_strength_edgelist_pairwise_sorted_norm.pkl'%dataset
with open(dict_name, 'wb') as f:
    pickle.dump(Dpairwise_n, f)

I build another dictionary which includes all higher orders interactions as triangles:

In [19]:
D3ho = D3.copy()

In [20]:
for clique in D4.keys():
    for triplet in itertools.combinations(clique,3): #all possible triplets in clique
        triplet = list(np.sort(triplet))
        if tuple(triplet) in D3ho:
            D3ho[tuple(triplet)] += 1
        else:
            D3ho[tuple(triplet)] = 1

for clique in D5.keys():
    for triplet in itertools.combinations(clique,3): #all possible triplets in clique
        triplet = list(np.sort(triplet))
        if tuple(triplet) in D3ho:
            D3ho[tuple(triplet)] += 1
        else:
            D3ho[tuple(triplet)] = 1


Normalize second order:

In [ ]:
D3ho_n = {}
for key in D3ho:
    D3ho_n[key] = D3ho[key]/max(Dpairwise.values())

Save second order:

In [21]:
dict_name = 'only_triangles/%s_strength_edgelist3_sorted_norm.pkl'%dataset
with open(dict_name, 'wb') as f:
    pickle.dump(D3ho, f)